In [ ]:
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
from phoenix.server.api.types.pagination import (
    NodeIdentifier,
    SortableField,
    SortableFieldType,
)

project_id = "UHJvamVjdDox"
client = Client(
    transport=RequestsHTTPTransport(url="http://127.0.0.1:6006/graphql", timeout=1),
    fetch_schema_from_transport=True,
)

In [ ]:
spans_query = gql(
    """query SpansQuery($projectId: GlobalID!, $after: String = null, $before: String = null, $filterCondition: String = null, $first: Int = null, $last: Int = null, $sort: SpanSort = null) {
  node(id: $projectId) {
    ... on Project {
      spans(
        after: $after
        before: $before
        filterCondition: $filterCondition
        first: $first
        last: $last
        rootSpansOnly: false
        sort: $sort
      ) {
        edges {
          cursor
        }
        pageInfo {
          hasNextPage
          hasPreviousPage
        }
      }
    }
  }
}"""
)

In [ ]:
# basic query
response = client.execute(
    spans_query,
    variable_values={"projectId": project_id, "first": 5},
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [765, 764, 763, 762, 761], new_ids

In [ ]:
# query with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": NodeIdentifier(rowid=761).to_cursor(),
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [760, 759, 758, 757, 756], new_ids

In [ ]:
# page ends on the penultimate record and excludees last record
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": NodeIdentifier(7).to_cursor(),
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
has_next_page = response["node"]["spans"]["pageInfo"]["hasNextPage"]
has_previous_page = response["node"]["spans"]["pageInfo"]["hasPreviousPage"]
assert new_ids == [6, 5, 4, 3, 2], new_ids
assert has_next_page is True
assert has_previous_page is False

In [ ]:
# page ends on the last record exactly
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": NodeIdentifier(6).to_cursor(),
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
has_next_page = response["node"]["spans"]["pageInfo"]["hasNextPage"]
has_previous_page = response["node"]["spans"]["pageInfo"]["hasPreviousPage"]
assert new_ids == [5, 4, 3, 2, 1], new_ids
assert has_next_page is False
assert has_previous_page is False

In [ ]:
# page ends before it reaches the limit
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": NodeIdentifier(5).to_cursor(),
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
has_next_page = response["node"]["spans"]["pageInfo"]["hasNextPage"]
has_previous_page = response["node"]["spans"]["pageInfo"]["hasPreviousPage"]
assert new_ids == [4, 3, 2, 1], new_ids
assert has_next_page is False
assert has_previous_page is False

In [ ]:
# basic filter condition
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "first": 5,
        "filterCondition": "span_kind == 'LLM'",
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [
    761,
    756,
    751,
    746,
    741,
], new_ids

In [ ]:
# basic filter condition with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "first": 5,
        "after": NodeIdentifier(
            761
        ).to_cursor(),  # skip the first span satisfying the filter condition
        "filterCondition": "span_kind == 'LLM'",
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [
    756,
    751,
    746,
    741,
    736,
], new_ids

In [ ]:
# compound filter condition with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "after": NodeIdentifier(
            761
        ).to_cursor(),  # skip the first span satisfying the filter condition
        "first": 5,
        "filterCondition": "span_kind == 'LLM' and cumulative_llm_token_count_prompt > 300",
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [
    756,
    751,
    736,
    731,
    721,
], new_ids

In [ ]:
# order by start time
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "desc"},
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [1, 2, 3, 4, 5], new_ids

In [ ]:
# order by cumulative prompt token count in descending order
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "cumulativeTokenCountPrompt", "dir": "desc"},
        "first": 5,
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [
    60,
    57,
    56,
    125,
    122,
], new_ids

In [ ]:
# order by descending start time with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "desc"},
        "first": 5,
        "after": NodeIdentifier(
            5,
            sortable_field=SortableField.from_stringified_value(
                type=SortableFieldType.DATETIME,
                stringified_value="2024-05-05T04:25:29.911245+00:00",
            ),
        ).to_cursor(),
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [6, 7, 8, 9, 10], new_ids

In [ ]:
# order by ascending start time with cursor
response = client.execute(
    spans_query,
    variable_values={
        "projectId": project_id,
        "sort": {"col": "startTime", "dir": "asc"},
        "first": 5,
        "after": NodeIdentifier(
            10,
            sortable_field=SortableField.from_stringified_value(
                type=SortableFieldType.DATETIME,
                stringified_value="2024-05-05T04:25:29.053197+00:00",
            ),
        ).to_cursor(),
    },
)
new_cursors = [edge["cursor"] for edge in response["node"]["spans"]["edges"]]
new_ids = [NodeIdentifier.from_cursor(cursor).rowid for cursor in new_cursors]
assert new_ids == [9, 8, 7, 6, 5], new_ids